# Gensim
They use Gensim as a baseline (200-dim vectors, 15 epochs).
## Usage
* [Doc2Vec tutorial](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb)

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from util import const, reader
import gensim
import sklearn.feature_extraction as sk_fe
import nltk

In [56]:
model_loc = const.Path.model('gensim_200.model')
read_model = True

def construct_doc2vec_model(data_loc, model_loc):
  def read_corpus(fname):
    with open(fname) as f:
      for i, line in enumerate(f):
        yield gensim.models.doc2vec.TaggedDocument(
          gensim.utils.simple_preprocess(line), [i])

  corpus = read_corpus(data_loc)
  model = gensim.models.Doc2Vec(
    corpus, size=200, workers=4, seed=const.SEED, iter=15)
  model.save(model_loc)
  return model

model = gensim.models.Doc2Vec.load(model_loc) if read_model\
  else construct_doc2vec_model(const.Path.TWEETS_RAW_TEXT, model_loc)

In [54]:
%%capture
tfidf_vectorizer = sk_fe.text.TfidfVectorizer(stop_words=nltk.stop_words)
content_transformed = tfidf_vectorizer.fit_transform(
  reader.stream_lines(const.Path.TWEETS_RAW_TEXT))

In [55]:
fst = content_transformed[0]
fst[fst.nonzero()]

matrix([[0.05193561, 0.34069949, 0.25497813, 0.2281689 , 0.35103557,
         0.11534646, 0.26594234, 0.2824803 , 0.19419119, 0.33055714,
         0.48087059, 0.10842513, 0.28861267, 0.06190617]])